# Data Pre-Processing

## Dealing with [*Missing* Data](https://en.wikipedia.org/wiki/Missing_data)

In most online _machine learning_ classes you are taught that when your data set is **incomplete** you can either:
1. Erase the corresponding rows with missing cells; or
2. _Impute_ (fill) the sample average of each column into those missing cells.

It turns out that the second option would require stronger assumptions than the first. If the observations are **missing completely at random** and your sample is i.i.d., the first option is harmless for large data sets.

## Encoding [Categorical Variables](https://en.wikipedia.org/wiki/Categorical_variable)

💻 Consider the ```hprice3``` data set from the ```wooldridge``` package:

In [1]:
%load_ext rpy2.ipython

In [2]:
## installing the 'wooldridge' package if not previously installed
%R if (!require(wooldridge)) install.packages('wooldridge')

%R data(hprice3)

## Obs:   321

##  1. year                     1978, 1981
##  2. age                      age of house
##  3. agesq                    age^2
##  4. nbh                      neighborhood, 0 to 6
##  5. cbd                      dist. to central bus. dstrct, feet
##  6. inst                     dist. to interstate, feet
##  7. linst                    log(inst)
##  8. price                    selling price
##  9. rooms                    # rooms in house
## 10. area                     square footage of house
## 11. land                     square footage lot
## 12. baths                    # bathrooms
## 13. dist                     dist. from house to incin., feet
## 14. ldist                    log(dist)
## 15. lprice                   log(price)
## 16. y81                      =1 if year = 1981
## 17. larea                    log(area)
## 18. lland                    log(land)
## 19. linstsq                  linst^2

R[write to console]: Loading required package: wooldridge



'hprice3'


In [3]:
import wooldridge as woo
hprice3 = woo.dataWoo('hprice3')

💻 Variables ```y81```, ```rooms```, and ```nbh``` are examples of <ins>categorical</ins> variables. In Econometrics, ```y81``` is called a standard dummy variable, ```rooms``` is called an _ordered_ categorical variable, ```nbh``` is called an _unordered_ categorical variable. Both ```rooms``` and ```nbh``` have _multiple_ categories. The fucntion ```as.factor()``` with option ```ordered=TRUE``` and ```ordered=FALSE``` (default) will allow us to handle them accordingly in all analysis.

In [4]:
%%R
attach(hprice3)
no.factor <- data.frame(y81=y81,rooms=rooms,nbh=nbh)
print(summary(no.factor))
detach(hprice3)

attach(hprice3)
yes.factor <- data.frame(y81=factor(y81),rooms=factor(rooms,ordered=TRUE),nbh=factor(nbh,ordered=FALSE))
print(summary(yes.factor))
detach(hprice3)

      y81             rooms             nbh       
 Min.   :0.0000   Min.   : 4.000   Min.   :0.000  
 1st Qu.:0.0000   1st Qu.: 6.000   1st Qu.:0.000  
 Median :0.0000   Median : 7.000   Median :2.000  
 Mean   :0.4424   Mean   : 6.586   Mean   :2.209  
 3rd Qu.:1.0000   3rd Qu.: 7.000   3rd Qu.:4.000  
 Max.   :1.0000   Max.   :10.000   Max.   :6.000  
 y81     rooms    nbh    
 0:179   4 :  4   0:121  
 1:142   5 : 30   1: 27  
         6 : 96   2: 47  
         7 :169   3:  7  
         8 : 10   4: 60  
         9 : 11   5: 27  
         10:  1   6: 32  


In [5]:
import pandas as pd
no_factor = hprice3[['y81','rooms','nbh']]
print(no_factor.dtypes)
yes_factor = pd.DataFrame({'y81':hprice3['y81'].astype('category'),
                           'rooms':hprice3['rooms'].astype('category'),
                           'nbh':hprice3['nbh'].astype('category')})
yes_factor.rooms.cat.set_categories(
    new_categories = sorted(hprice3['rooms'].unique()), ordered = True, inplace = True)
print(yes_factor.dtypes)

y81      int64
rooms    int64
nbh      int64
dtype: object
y81      category
rooms    category
nbh      category
dtype: object


💻 The default behavior in regression is to transformed ordered and unordered categorical variable with multiple categories into a set of $c-1$ dummy variables and include them as regressors, where $c$ represents the number of categories.

In [6]:
%%R
ols <- lm(lprice ~ lland + larea + I(log(cbd)) +as.factor(y81) + as.factor(rooms) + as.factor(nbh) +linst + linstsq + ldist + baths + age + agesq,data=hprice3)

## installing the 'lmtest', 'sandwich' packages if not previously installed
if (!require(lmtest)) install.packages('lmtest')
if (!require(sandwich)) install.packages('sandwich')

## turning 'off' scientific notation
options(scipen = 999) 

## calculating standard t-statistics for 'significance'
print(coeftest(ols, vcov = vcovHC, type = 'HC1'))

R[write to console]: Loading required package: lmtest

R[write to console]: Loading required package: zoo

R[write to console]: 
Attaching package: ‘zoo’


R[write to console]: The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


R[write to console]: Loading required package: sandwich




t test of coefficients:

                       Estimate   Std. Error t value              Pr(>|t|)    
(Intercept)         3.497413578  2.266536887  1.5431             0.1238762    
lland               0.122178750  0.037345254  3.2716             0.0011950 ** 
larea               0.353527029  0.064960239  5.4422            0.00000011 ***
I(log(cbd))        -0.030909378  0.057676152 -0.5359             0.5924187    
as.factor(y81)1     0.379015730  0.024474885 15.4859 < 0.00000000000000022 ***
as.factor(rooms)5   0.132637038  0.156849504  0.8456             0.3984363    
as.factor(rooms)6   0.061507499  0.158817991  0.3873             0.6988232    
as.factor(rooms)7   0.140493063  0.165317814  0.8498             0.3960984    
as.factor(rooms)8   0.173819280  0.167557064  1.0374             0.3004028    
as.factor(rooms)9   0.290598039  0.183811551  1.5810             0.1149488    
as.factor(rooms)10  0.393247317  0.215237037  1.8270             0.0686934 .  
as.factor(nbh)1    -0.0305

In [7]:
from statsmodels.formula.api import ols
from numpy import log
f = 'lprice ~ lland + larea + log(cbd) + C(y81) + C(rooms) + C(nbh) + linst + linstsq + ldist + baths + age + agesq'
res = ols(formula=f,data=hprice3).fit(cov_type='HC1')
print(res.summary().tables[1])

                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          3.4974      2.267      1.543      0.123      -0.945       7.940
C(y81)[T.1]        0.3790      0.024     15.486      0.000       0.331       0.427
C(rooms)[T.5]      0.1326      0.157      0.846      0.398      -0.175       0.440
C(rooms)[T.6]      0.0615      0.159      0.387      0.699      -0.250       0.373
C(rooms)[T.7]      0.1405      0.165      0.850      0.395      -0.184       0.465
C(rooms)[T.8]      0.1738      0.168      1.037      0.300      -0.155       0.502
C(rooms)[T.9]      0.2906      0.184      1.581      0.114      -0.070       0.651
C(rooms)[T.10]     0.3932      0.215      1.827      0.068      -0.029       0.815
C(nbh)[T.1]       -0.0305      0.061     -0.498      0.619      -0.151       0.090
C(nbh)[T.2]       -0.0568      0.032     -1.776      0.076      -0.120       0.006
C(nb

📌 ````Python````: You can also change the _base category_ (the category left out):

In [8]:
f_b = 'lprice ~ lland + larea + log(cbd) + C(y81,Treatment(1)) + C(rooms,Treatment(7)) + C(nbh,Treatment(3)) + linst + linstsq + ldist + baths + age + agesq'
res_b = ols(formula=f_b,data=hprice3).fit(cov_type='HC1')
print(res_b.summary().tables[1])

                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        3.8180      2.269      1.683      0.092      -0.629       8.265
C(y81, Treatment(1))[T.0]       -0.3790      0.024    -15.486      0.000      -0.427      -0.331
C(rooms, Treatment(7))[T.4]     -0.1405      0.165     -0.850      0.395      -0.465       0.184
C(rooms, Treatment(7))[T.5]     -0.0079      0.046     -0.171      0.864      -0.098       0.082
C(rooms, Treatment(7))[T.6]     -0.0790      0.035     -2.233      0.026      -0.148      -0.010
C(rooms, Treatment(7))[T.8]      0.0333      0.047      0.702      0.483      -0.060       0.126
C(rooms, Treatment(7))[T.9]      0.1501      0.078      1.918      0.055      -0.003       0.303
C(rooms, Treatment(7))[T.10]     0.2528      0.145      1.741      0.082      -0.032       0.537
C(nbh, Treatment(3))[T.0]     

💻 In many machine learning algorithms you are required to provide the design (model) matrix, $\mathbf{X}$ (*without* and intercept), and response vector, $\mathbf{y}$.

**<span style="color:green">Exercise:</span>**: ```R``` - Use the ```model.matrix``` function to extract the design matrix _without_ an intercept from the previously created ```ols``` object and verify it contains 22 columns of regressors/features.

In [9]:
%%R
X <- model.matrix(ols)[,-1]
print(dim(X))
print(colnames(X))

[1] 321  22
 [1] "lland"              "larea"              "I(log(cbd))"       
 [4] "as.factor(y81)1"    "as.factor(rooms)5"  "as.factor(rooms)6" 
 [7] "as.factor(rooms)7"  "as.factor(rooms)8"  "as.factor(rooms)9" 
[10] "as.factor(rooms)10" "as.factor(nbh)1"    "as.factor(nbh)2"   
[13] "as.factor(nbh)3"    "as.factor(nbh)4"    "as.factor(nbh)5"   
[16] "as.factor(nbh)6"    "linst"              "linstsq"           
[19] "ldist"              "baths"              "age"               
[22] "agesq"             


In [10]:
import patsy
y,X = patsy.dmatrices(f, hprice3, return_type='dataframe')
del X['Intercept']
print(X.shape)
print(list(X.columns))

(321, 22)
['C(y81)[T.1]', 'C(rooms)[T.5]', 'C(rooms)[T.6]', 'C(rooms)[T.7]', 'C(rooms)[T.8]', 'C(rooms)[T.9]', 'C(rooms)[T.10]', 'C(nbh)[T.1]', 'C(nbh)[T.2]', 'C(nbh)[T.3]', 'C(nbh)[T.4]', 'C(nbh)[T.5]', 'C(nbh)[T.6]', 'lland', 'larea', 'log(cbd)', 'linst', 'linstsq', 'ldist', 'baths', 'age', 'agesq']


📌 ```R```: It is good practice to define categorical variables _outside_ the model formula/fitting. When doing this, one can easily change the 'base' category using the ```relevel()``` function along with the ```within()``` function.

## Including [Interaction Terms](https://en.wikipedia.org/wiki/Interaction_(statistics) )

In the previously fitted model we included ```linstsq``` and ```agesq``` as predictors. These correspond to the squared of the original predictors ```linst``` and ```age```. In economics we include such predictors to account for increasing/decreasing returns to scale in modelling. Since $\texttt{linst}^2=\texttt{linst}\times\texttt{linst}$ and $\texttt{age}^2=\texttt{age}\times\texttt{age}$, one can think of them as a specific type of interaction terms (products with themselves).

<ins>Example</ins>: Consider the following version of the previously estimated model

$$
\begin{aligned}
\texttt{lprice}&=\beta_0+\beta_1\texttt{lland}+\beta_2\texttt{larea}+ \beta_3\texttt{linst}+\beta_4\texttt{age}\\
               & + \beta_5\texttt{linst}\times\texttt{age}+\beta_6\texttt{linst}^2+\beta_7\texttt{age}^2+e
\end{aligned}
$$

In [11]:
%%R

## OLS: estimating this new model
ols.0 <- lm(lprice ~ lland + larea + linst + age + I(linst*age) + linstsq + agesq,data=hprice3)
print(round(coef(ols.0),5))

   (Intercept)          lland          larea          linst            age 
       2.25472        0.10212        0.63839        0.85279       -0.01741 
I(linst * age)        linstsq          agesq 
       0.00074       -0.05226        0.00004 


In [12]:
f = 'lprice ~ lland + larea + linst + age + I(linst*age) + linstsq + agesq'
res_0 = ols(formula=f,data=hprice3).fit()

import numpy as np
np.round(res_0.params,5)

Intercept         2.25472
lland             0.10212
larea             0.63839
linst             0.85279
age              -0.01741
I(linst * age)    0.00074
linstsq          -0.05226
agesq             0.00004
dtype: float64

By defining $\mathbf{x}=[\texttt{lland},\texttt{larea},\texttt{linst},\texttt{age}]^\prime$, and Assumption MLR.4, one has

$$
\frac{\partial E[\texttt{lprice}|\mathbf{x}]}{\partial \texttt{linst}}=\beta_3+\beta_5\texttt{age}+2\beta_6\texttt{linst}.
$$

Then $\beta_3$ represents the average price elasticty with respect to the distance from the highway for a home that is brand new ($\texttt{age}=0$) and that is located 1 feet away from the highway ($\texttt{linst}=\log(\texttt{inst})=0$, i.e., $\texttt{inst}=1$).

💻 Calculating the summary statistics for variables ```age``` and ```inst``` in the data set.

In [13]:
%%R

## printing summary statistics for 'age' and 'inst'
print(summary(subset(hprice3,select=c('age','inst','linst'))))

      age              inst           linst       
 Min.   :  0.00   Min.   : 1000   Min.   : 6.908  
 1st Qu.:  0.00   1st Qu.: 9000   1st Qu.: 9.105  
 Median :  4.00   Median :16000   Median : 9.680  
 Mean   : 18.01   Mean   :16442   Mean   : 9.481  
 3rd Qu.: 22.00   3rd Qu.:24000   3rd Qu.:10.086  
 Max.   :189.00   Max.   :34000   Max.   :10.434  


In [14]:
hprice3[['age','inst','linst']].describe()

,age,inst,linst
count,321.000000,321.000000,321.000000
mean,18.009346,16442.367601,9.480513
std,32.565845,9033.130652,0.777165
min,0.000000,1000.000000,6.907800
25%,0.000000,9000.000000,9.105000
50%,4.000000,16000.000000,9.680300
75%,22.000000,24000.000000,10.086000
max,189.000000,34000.000000,10.434000


Now consider the following alternative specification

$$
\begin{aligned}
\texttt{lprice}&=\delta_0+\beta_1\texttt{lland}+\beta_2\texttt{larea}+ \delta_3\texttt{linst}+\delta_4\texttt{age}\\
               & + \beta_5[\texttt{linst}-\mu_{\texttt{linst}}]\times[\texttt{age}-\mu_{\texttt{age}}]+\beta_6[\texttt{linst}-\mu_{\texttt{linst}}]^2+\beta_7[\texttt{age}-\mu_{\texttt{age}}]^2+e.
\end{aligned}
$$

In this case

$$
\frac{\partial E[\texttt{lprice}|\mathbf{x}]}{\partial \texttt{linst}}=\delta_3+\beta_5[\texttt{age}-\mu_{\texttt{age}}]+2\beta_6[\texttt{linst}-\mu_{\texttt{linst}}].
$$

Then $\delta_3$ represents the average price elasticty with respect to the distance from the highway for a home that is 18 years old ($\widehat{\mu}_{\texttt{age}}\approx 18.01$) and that is located 16,442 feet away from the highway ($\widehat{\mu}_{\texttt{inst}}\approx 16,442$).

In [15]:
## OLS: estimating this new 'version' model
%R ols.1 <- lm(lprice ~ lland + larea + linst + age + I((linst-log(mean(inst)))*(age-mean(age))) + I((linst-log(mean(inst)))^2) + I((age-mean(age))^2),data=hprice3)

## printing ols results up to 5 decimals
%R round(data.frame(ols.0=coef(ols.0),ols.1=coef(ols.1)),5)

,ols.0,ols.1
(Intercept),2.25472,7.03626
lland,0.10212,0.10212
larea,0.63839,0.63839
linst,0.85279,-0.14856
age,-0.01741,-0.00865
I(linst * age),0.00074,0.00074
linstsq,-0.05226,-0.05226
agesq,0.00004,0.00004


In [16]:
m = hprice3[['age','inst','linst']].mean()
f = 'lprice ~ lland + larea + linst + age + I((linst-log(m.loc["inst"]))*(age-m.loc["age"])) + I((linst-log(m.loc["inst"]))**2) + I((age-m.loc["age"])**2)'
res_1 = ols(formula=f,data=hprice3).fit()

import numpy as np
pd.DataFrame({'ols.0':res_0.params,'ols.1':res_1.params})

,ols.0,ols.1
"I((age - m.loc[""age""]) ** 2)",NaN,0.000044
"I((linst - log(m.loc[""inst""])) * (age - m.loc[""age""]))",NaN,0.000740
"I((linst - log(m.loc[""inst""])) ** 2)",NaN,-0.052262
I(linst * age),0.000740,NaN
Intercept,2.254722,7.036262
age,-0.017408,-0.008652
agesq,0.000044,NaN
larea,0.638386,0.638386
linst,0.852794,-0.148562
linstsq,-0.052262,NaN


⁉️ Is it a coincidence that some of the estimates are the same in both specifications? provide an algebraic explanation.

## Beta Coefficients

Why is standardization useful? It is easiest to start with the original OLS equation, with the variables in their original forms:

$$
y_{i}=\widehat{\beta}_{0}+\widehat{\beta}_{1} x_{i 1}+\widehat{\beta}_{2} x_{i 2}+\ldots+\widehat{\beta}_{k} x_{i k}+\hat{e}_{i}.
$$

We have included the observation subscript i to emphasize that our standardization is applied to all sample values. If we average the previous equation and use the fact that $\{\widehat{e}_i:i=1,\ldots,n\}$ has sample mean zero one has

$$
y_{i}-\overline{y}=\widehat{\beta}_{1}\left(x_{i 1}-\overline{x}_{1}\right)+\widehat{\beta}_{2}\left(x_{i 2}-\overline{x}_{2}\right)+\ldots+\widehat{\beta}_{k}\left(x_{i k}-\overline{x}_{k}\right)+\widehat{e}_{i}.
$$

Now, let $\widehat{\sigma}_{y}$ be the sample standard deviation for the dependent variable, let $\widehat{\sigma}_{1}$ be the sample sd for $x_{1}$, let $\widehat{\sigma}_{2}$ be the sample standard deviation, ```pandas.DataFrame.std()```, for $x_{2}$, and so on. Then, simple algebra gives the equation

$$
\begin{aligned}
\left(y_{i}-\overline{y}\right) / \widehat{\sigma}_{y} &=\left(\widehat{\sigma}_{1} / \widehat{\sigma}_{y}\right) \widehat{\beta}_{1}\left[\left(x_{i 1}-\overline{x}_{1}\right) / \widehat{\sigma}_{1}\right]+\ldots \\ &+\left(\widehat{\sigma}_{k} / \widehat{\sigma}_{y}\right) \widehat{\beta}_{k}\left[\left(x_{i k}-\overline{x}_{k}\right) / \widehat{\sigma}_{k}\right]+\left(\widehat{e}_{i} / \widehat{\sigma}_{y}\right).
\end{aligned}
$$

Now define $z_{i y}=\left(y_{i}-\overline{y}\right) / \widehat{\sigma}_{y}$, $z_{i l}=\left[\left(x_{i l}-\overline{x}_{l}\right) / \widehat{\sigma}_{l}\right]$, $\widehat{b}_{l}=\left(\widehat{\sigma}_{l} / \widehat{\sigma}_{y}\right) \widehat{\beta}_{l}$ for $l=1,\ldots,k$, and $\widehat{\varepsilon}_i=\left(\widehat{e}_{i} / \widehat{\sigma}_{y}\right)$. Then

$$
z_{y}=\widehat{b}_{1} z_{1}+\widehat{b}_{2} z_{2}+\ldots+\widehat{b}_{k} z_{k}+\widehat{\varepsilon}_i.
$$

The new coefficients are

$$
\widehat{b}_{j}=\left(\widehat{\sigma}_{j} / \widehat{\sigma}_{y}\right) \widehat{\beta}_{j} \text { for } j=1, \ldots, k.
$$

These $\widehat{b}_{j}$ are traditionally called **standardized coefficients** or **beta coefficients**.

✍🏽 If $x_l$ increases by one standard deviation, then $\widehat{y}$ changes by $\widehat{b}_l$ standard deviations. Thus, *we are measuring effects not in terms of the original units of $y$ or the $x_l$, but in standard deviation units*.

In [17]:
%%R

## installing the 'lm.beta' package if not previously installed
if (!require(lm.beta)) install.packages('lm.beta')

print(coef(lm.beta(ols.1)))

R[write to console]: Loading required package: lm.beta



                                     (Intercept) 
                                      0.00000000 
                                           lland 
                                      0.18685413 
                                           larea 
                                      0.49640630 
                                           linst 
                                     -0.26349631 
                                             age 
                                     -0.64300498 
I((linst - log(mean(inst))) * (age - mean(age))) 
                                      0.05000987 
                  I((linst - log(mean(inst)))^2) 
                                     -0.15421830 
                          I((age - mean(age))^2) 
                                      0.37490499 


In [18]:
import patsy
f = 'lprice ~ lland + larea + linst + age + I((linst-log(m.loc["inst"]))*(age-m.loc["age"])) + I((linst-log(m.loc["inst"]))**2) + I((age-m.loc["age"])**2) - 1'
y,X = patsy.dmatrices(f, hprice3, return_type='dataframe')

import statsmodels.api as sm
from scipy.stats.mstats import zscore

print(sm.OLS(zscore(y),zscore(X)).fit().params)

[ 0.18685413  0.4964063  -0.26349631 -0.64300498  0.05000987 -0.1542183
  0.37490499]


💻 Since each $x_l$ have been standardized, comparing the _magnitudes_ of the resulting beta coefficients is now useful. The ```age``` of the house seems to have the largest effect on the price of a home.

<ins>Pre-Processing in Machine Learning</ins>: In machine learning, standardizing (*re-centering* and *scaling*) predictors is commonly done before model fitting. No transformation is usually done to the outcome variable.

In [19]:
%%R

X1 <- model.matrix(ols.1)[,-1]
datos <- cbind(data.frame(lprice=hprice3$lprice),as.data.frame(X1))

## installing the 'caret' package if not previously installed
if (!require(caret)) install.packages('caret')

modelo <- train(lprice ~ .,data = datos,method = 'lm',preProcess = c('scale', 'center'))

print(coef(modelo$finalModel))

R[write to console]: Loading required package: caret

R[write to console]: Loading required package: lattice

R[write to console]: Loading required package: ggplot2



                                             (Intercept) 
                                             11.37811794 
                                                   lland 
                                              0.08187470 
                                                   larea 
                                              0.21751254 
                                                   linst 
                                             -0.11545734 
                                                     age 
                                             -0.28174834 
`\\`I((linst - log(mean(inst))) * (age - mean(age)))\\`` 
                                              0.02191305 
                  `\\`I((linst - log(mean(inst)))^2)\\`` 
                                             -0.06757452 
                          `\\`I((age - mean(age))^2)\\`` 
                                              0.16427378 


In [20]:
print(sm.OLS(y,np.c_[np.ones(X.shape[0]),zscore(X)]).fit().params)

const    11.378118
x1        0.081747
x2        0.217173
x3       -0.115277
x4       -0.281309
x5        0.021879
x6       -0.067469
x7        0.164018
dtype: float64
